## **Notebook for gene target predictions to increase TAG production**

By Christina Schenk

Tested on biodesign_3.7 kernel on jprime

This notebook predicts interventions to increase TAG production for R. opacus cultures growing with phenol and glucose.

**Import python packages**

In [1]:
import cobra
import cplex
import sys
OptForceDir = '../OptForce'
sys.path.append(OptForceDir)
from optforce import optforce

## R.opacus TAG production

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    model.objective = 'Growth_Glucose'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    display('Original TAG Production', TAGprod)
    display('*********************')

### **1 Knockout Glucose**

In [2]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
#     model.objective = 'Growth_Glucose'
#     fbasol = model.optimize()
#     TAGprod = fbasol.fluxes['tag_production']
#     display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

'*********************'

In [3]:
display(oursolution)

{'Must_solution': {'MustL': ['PPNDH', 'G3PD1', 'NNDPR', 'ASPO5', 'QULNS'],
  'MustU': ['G3PAT160', 'AGPAT160', 'PHETA1'],
  'MustLL': [['DB4PS', 'RBFSa'],
   ['DB4PS', 'GCALDD'],
   ['RBFSa', 'GCALDD'],
   ['DNMPPA', 'GCALDD'],
   ['ADCL', 'RBFSa'],
   ['DNMPPA', 'DB4PS'],
   ['GTPCI', 'DB4PS'],
   ['ADCS', 'RBFSa'],
   ['ADCL', 'DB4PS'],
   ['ADCS', 'GCALDD'],
   ['DHFS', 'RBFSa'],
   ['DNTPPA', 'GCALDD'],
   ['DNTPPA', 'RBFSa'],
   ['DHFS', 'GCALDD'],
   ['GTPCI', 'GCALDD'],
   ['DB4PS', 'ADCS'],
   ['DB4PS', 'DNTPPA'],
   ['DHFS', 'DB4PS'],
   ['ADCL', 'GCALDD'],
   ['DNMPPA', 'RBFSa'],
   ['GTPCI', 'RBFSa'],
   ['DPCOAK', 'RBFSa'],
   ['PNTK', 'RBFSa'],
   ['DPCOAK', 'GCALDD'],
   ['PPCDC', 'RBFSa'],
   ['DB4PS', 'PNTK'],
   ['DPCOAK', 'DB4PS'],
   ['PTPATi', 'RBFSa'],
   ['PTPATi', 'GCALDD'],
   ['PPCDC', 'DB4PS'],
   ['PPCDC', 'GCALDD'],
   ['PNTK', 'GCALDD'],
   ['PTPATi', 'DB4PS']],
  'MustUU': [],
  'MustLU': []},
 'Force_solution': [{'status': 101,
   'objective value': 14.56

In [2]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
#     model.objective = 'Growth_Glucose'
#     fbasol = model.optimize()
#     TAGprod = fbasol.fluxes['tag_production']
#     display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'EX_tag'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

'*********************'

In [3]:
display(oursolution)

{'Must_solution': {'MustL': ['PPNDH', 'G3PD1', 'NNDPR', 'ASPO5', 'QULNS'],
  'MustU': ['G3PAT160', 'AGPAT160', 'PHETA1'],
  'MustLL': [['DB4PS', 'RBFSa'],
   ['DB4PS', 'GCALDD'],
   ['RBFSa', 'GCALDD'],
   ['GTPCI', 'GCALDD'],
   ['ADCL', 'GCALDD'],
   ['ADCS', 'GCALDD'],
   ['DNMPPA', 'DB4PS'],
   ['GTPCI', 'RBFSa'],
   ['DHFS', 'GCALDD'],
   ['DB4PS', 'GTPCI'],
   ['DNMPPA', 'GCALDD'],
   ['DNTPPA', 'GCALDD'],
   ['DNMPPA', 'RBFSa'],
   ['DHFS', 'RBFSa'],
   ['RBFSa', 'DNTPPA'],
   ['DB4PS', 'ADCS'],
   ['ADCS', 'RBFSa'],
   ['RBFSa', 'ADCL'],
   ['DB4PS', 'DHFS'],
   ['DB4PS', 'ADCL'],
   ['DB4PS', 'DNTPPA'],
   ['PPCDC', 'GCALDD'],
   ['DPCOAK', 'RBFSa'],
   ['RBFSa', 'PPCDC'],
   ['DB4PS', 'PNTK'],
   ['RBFSa', 'PNTK'],
   ['PNTK', 'GCALDD'],
   ['PTPATi', 'GCALDD'],
   ['DB4PS', 'DPCOAK'],
   ['DPCOAK', 'GCALDD'],
   ['DPR', 'PTPATi', 'RBFSa'],
   ['DB4PS', 'PTPATi'],
   ['DB4PS', 'PPCDC']],
  'MustUU': [],
  'MustLU': []},
 'Force_solution': [{'status': 101,
   'objective value'

### **1 Knockout Phenol**

In [2]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='phenol'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    model.objective='Growth_Phenol'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

'Original TAG Production'

0.0

'*********************'

In [3]:
display(oursolution)

{'Must_solution': {'MustL': ['G3PD1',
   'FACOAL150_anteiso',
   'KAS4',
   'OIVD1r',
   'KAS3',
   'FACOAL150_ISO',
   'KAS12',
   'FACOAL170_anteiso',
   'NNDPR',
   'QULNS',
   'ASPO5',
   'FACOAL170_ISO',
   'KAS11'],
  'MustU': ['AGPAT160', 'G3PAT160'],
  'MustLL': [['FACOAL160_ISO', 'KAS6'],
   ['KAS6', 'RBFSa'],
   ['FACOAL160_ISO', 'RBFSa'],
   ['KAS6', 'DB4PS'],
   ['FACOAL160_ISO', 'DB4PS'],
   ['KAS6', 'GCALDD'],
   ['FACOAL160_ISO', 'GCALDD'],
   ['ADCL', 'KAS6'],
   ['FACOAL160_ISO', 'ADCS'],
   ['KAS6', 'ADCS'],
   ['FACOAL160_ISO', 'DHFS'],
   ['FACOAL160_ISO', 'ADCL'],
   ['FACOAL160_ISO', 'GTPCI'],
   ['FACOAL160_ISO', 'DNMPPA'],
   ['KAS6', 'DHFS'],
   ['KAS6', 'GTPCI'],
   ['FACOAL160_ISO', 'DNTPPA'],
   ['KAS6', 'DNTPPA'],
   ['KAS6', 'DNMPPA'],
   ['RBFSa', 'GCALDD'],
   ['DB4PS', 'GCALDD'],
   ['DB4PS', 'RBFSa'],
   ['FACOAL160_ISO', 'PNTK'],
   ['KAS6', 'PTPATi'],
   ['PNTK', 'KAS6'],
   ['FACOAL160_ISO', 'PTPATi'],
   ['KAS6', 'DPCOAK'],
   ['FACOAL160_ISO', 'DP

In [2]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='phenol'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    model.objective='Growth_Phenol'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'EX_tag'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 1
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

'Original TAG Production'

0.0

'*********************'

In [3]:
display(oursolution)

{'Must_solution': {'MustL': ['G3PD1',
   'KAS4',
   'FACOAL150_anteiso',
   'OIVD1r',
   'FACOAL150_ISO',
   'KAS3',
   'KAS12',
   'FACOAL170_anteiso',
   'ASPO5',
   'NNDPR',
   'QULNS',
   'KAS11',
   'FACOAL170_ISO'],
  'MustU': ['G3PAT160', 'AGPAT160'],
  'MustLL': [['KAS6', 'FACOAL160_ISO'],
   ['GCALDD', 'KAS6'],
   ['GCALDD', 'FACOAL160_ISO'],
   ['RBFSa', 'FACOAL160_ISO'],
   ['KAS6', 'RBFSa'],
   ['FACOAL160_ISO', 'DB4PS'],
   ['KAS6', 'DB4PS'],
   ['ADCL', 'FACOAL160_ISO'],
   ['KAS6', 'ADCS'],
   ['FACOAL160_ISO', 'GTPCI'],
   ['KAS6', 'DHFS'],
   ['DHFS', 'FACOAL160_ISO'],
   ['ADCS', 'FACOAL160_ISO'],
   ['KAS6', 'GTPCI'],
   ['ADCL', 'KAS6'],
   ['FACOAL160_ISO', 'DNMPPA'],
   ['KAS6', 'DNMPPA'],
   ['KAS6', 'DNTPPA'],
   ['FACOAL160_ISO', 'DNTPPA'],
   ['GCALDD', 'DB4PS'],
   ['RBFSa', 'DB4PS'],
   ['GCALDD', 'RBFSa'],
   ['KAS6', 'PNTK'],
   ['PTPATi', 'KAS6'],
   ['FACOAL160_ISO', 'DPCOAK'],
   ['FACOAL160_ISO', 'PNTK'],
   ['KAS6', 'DPCOAK'],
   ['KAS6', 'PPCDC'],
  

### **2 Knockouts Glucose**

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='glucose'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
#     model.objective = 'Growth_Glucose'
#     fbasol = model.optimize()
#     TAGprod = fbasol.fluxes['tag_production']
#     display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 2
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

19.262947549367595


'max target'

36.240310077519354

In [ ]:
display(oursolution)

### **2 Knockouts Phenol**

In [ ]:
#file_name =  '../GSMs/Ropacus_annotated_curated_with_phenol_custom_biomass.xml'
file_name =  '../GSMs/Ropacus_annotated_curated.xml'
model = cobra.io.read_sbml_model(file_name)

with model:
    medium = model.medium
    substrate='phenol'
    if substrate=='phenol':
        #model.objective = 'Growth_Phenol'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Glucose').upper_bound = 0
        model.reactions.get_by_id('Growth_Glucose').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        #medium["EX_glc__D_e"] = 0
        medium["EX_phenol_e"] = 100#sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_phenol_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_glc__D_e').upper_bound = 0
        model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0
        biomass = 'Growth'
        #medium["EX_phenol_e"] = sub_uptake_rate
    elif substrate=='glucose':
        #model.objective = 'Growth_Glucose'
        medium = {key:1000 for (key,value) in model.medium.items()}
        model.reactions.get_by_id('Growth_Phenol').upper_bound = 0
        model.reactions.get_by_id('Growth_Phenol').lower_bound = 0
        #model.reactions.get_by_id('Growth').upper_bound = 0
        #model.reactions.get_by_id('Growth').lower_bound = 0
        medium["EX_glc__D_e"] = 100#sub_uptake_rate
        #medium["EX_phenol_e"] = 0
        #model.reactions.get_by_id('EX_glc__D_e').upper_bound = -sub_uptake_rate
        #model.reactions.get_by_id('EX_glc__D_e').lower_bound = -sub_uptake_rate
        model.reactions.get_by_id('EX_phenol_e').upper_bound = 0
        model.reactions.get_by_id('EX_phenol_e').lower_bound = 0
        biomass = 'Growth'
    else:
        print('Unknown substrate: Please choose among phenol and glucose')
    model.medium = medium
    model.objective='Growth_Phenol'
    fbasol = model.optimize()
    TAGprod = fbasol.fluxes['tag_production']
    display('Original TAG Production', TAGprod)
    display('*********************')
    target = 'tag_production'#'EX_acac_e'#EX_ac_e'#'EX_octa_e'# #acac works!!!'EX_dca_e'#
    maximum_knockout = 2
    minimum_growth_fraction = 0.1 #0.1 Paper
    minimum_target_fraction = 0.75 #0.9 
    high_order_must = False
    number_of_solutions = 10
    time_limit=180
#     if 'cplex' in cobra.util.solvers:
    model.solver = 'cplex'
    oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
             high_order_must, number_of_solutions, time_limit)#, verbose=False)
#     if 'gurobi' in cobra.util.solvers:
#         model.solver = 'gurobi'
#         oursolution = optforce(model, biomass, target, maximum_knockout, minimum_growth_fraction, minimum_target_fraction,
#                  high_order_must, number_of_solutions, time_limit)#, verbose=True)

In [ ]:
display(oursolution)